In [1]:
from classes.Utils import *
from classes.CRV import *
from classes.Corpus import *
from classes.Vectorizer import *

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
corpus = Corpus(
    filepath = 'corpora/recipes',
    spell_correct = 'corpora/spelling_dictionary',
    lemmatize = False,
    removal_threshold = 20)

vectorizer = corpus.create_vectorizer(removal_threshold = 20)

Data collected:
    - 1388 unique words found.
    - most common words: ['\n', '-', '.', '1', ',']
    - least common words: ['splash', 'step', 'teriyaki', 'there', 'translucent']


Corpus loaded:
    - 5000 sentences.
    - longest sentence: 

<START> whole wheat yeast bread 
 
 ingredients : 
 - 1 cup ( 2 2 5 ml ) <UNK> milk 
 - 1 / 4 cup ( 6 0 ml ) <UNK> 
 - 4 tsp ( 2 0 ml ) . salt 
 - 1 / 4 cup ( 6 0 ml ) honey 
 - 1 cup ( 2 2 5 ml ) water , 1 1 0 - 1 1 5 degrees 
 - 2 pkg . dry yeast 
 - 2 tbsp ( 3 0 ml ) . brown sugar 
 - 2 cups ( 4 7 5 ml ) whole wheat flour 
 - 3 cups ( 7 0 0 ml ) or more white flour 
 
 directions : 
 - mix <UNK> milk , <UNK> , salt & honey and let cool to lukewarm - about 8 5 degrees ( 3 0 c . ) . 
 - mix water , yeast & brown sugar and let stand to dissolve for 2 - 3 minutes . 
 - add yeast mixture to milk mixture . 
 - add whole wheat flour and enough white flour to make dough . 
 - mix well . 
 - knead 5 - 1 0 minutes . 
 - put in bowl to rise until double

In [4]:
twentieth = vectorizer.vsize // 20
differences = np.zeros((vectorizer.vsize, vectorizer.vsize,))
for i in range(vectorizer.vsize):
    word_diffs = keras.ops.einsum('v,bv->bv', vectorizer.matrix[i], vectorizer.matrix)
    word_diffs = keras.ops.sqrt(word_diffs)
    word_diffs = keras.ops.sum(word_diffs, axis = -1)
    word_diffs *= word_diffs
    differences[i] = word_diffs.numpy()
    
    if (i % twentieth == 0):
        print('twentieth')

differences *= (1 - np.eye(differences.shape[0], differences.shape[1]))

twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth
twentieth


In [5]:
print(differences[
    vectorizer.embedding_indices['cinnamon'],
    vectorizer.embedding_indices['nutmeg']])

print(differences[
    vectorizer.embedding_indices['cinnamon'],
    vectorizer.embedding_indices['milk']])

0.7801078039232636
0.5565421987725115


In [6]:
print(np.unravel_index(differences.argmax(), differences.shape))

(np.int64(6), np.int64(12))


In [7]:
print(corpus.vocab[6], corpus.vocab[12])

2 4


In [8]:
print(vectorizer.to_CRV('2'))
print(vectorizer.to_CRV('4'))

{ + 0.13⋅1 + 0.12⋅- + 0.11⋅/ + 0.1⋅\n + 0.09⋅. + 0.04⋅c + 0.02⋅cup + 0.02⋅tsp + 0.01⋅0 + 0.01⋅cups + 0.01⋅tbsp + 0.01⋅tablespoons + 0.01⋅minutes + 0.01⋅3 + 0.01⋅5 + 0.01⋅( + 0.01⋅to + 0.01⋅<UNK> + 0.01⋅teaspoon + 0.01⋅for + 962 others}
{ + 0.13⋅/ + 0.12⋅1 + 0.09⋅. + 0.08⋅- + 0.05⋅\n + 0.05⋅c + 0.04⋅3 + 0.04⋅cup + 0.03⋅0 + 0.02⋅5 + 0.02⋅to + 0.02⋅tsp + 0.02⋅teaspoon + 0.02⋅minutes + 0.01⋅( + 0.01⋅<UNK> + 0.01⋅cups + 0.01⋅for + 0.01⋅2 + 0.01⋅oz + 728 others}


In [9]:
positions = np.random.uniform(0.001, 1, (vectorizer.vsize, 2))

lr = .1

In [10]:
differences = 1 - differences

In [15]:

for n in range(500):
    # 0wd - w0d = wod (word, to-other, dimension)
    dirs_towards = np.expand_dims(positions, 0) - np.expand_dims(positions, 1)
    # wo
    dists = np.sqrt(np.sum(dirs_towards**2, axis = -1))
    # wo
    change_amts = (dists - differences**3) / (dists + .000001)

    vecs = np.einsum('wod,wo->wd', dirs_towards, change_amts) / vectorizer.vsize

    positions += vecs * lr

    print(f'{n} done!')


0 done!
1 done!
2 done!
3 done!
4 done!
5 done!
6 done!
7 done!
8 done!
9 done!
10 done!
11 done!
12 done!
13 done!
14 done!
15 done!
16 done!
17 done!
18 done!
19 done!
20 done!
21 done!
22 done!
23 done!
24 done!
25 done!
26 done!
27 done!
28 done!
29 done!
30 done!
31 done!
32 done!
33 done!
34 done!
35 done!
36 done!
37 done!
38 done!
39 done!
40 done!
41 done!
42 done!
43 done!
44 done!
45 done!
46 done!
47 done!
48 done!
49 done!
50 done!
51 done!
52 done!
53 done!
54 done!
55 done!
56 done!
57 done!
58 done!
59 done!
60 done!
61 done!
62 done!
63 done!
64 done!
65 done!
66 done!
67 done!
68 done!
69 done!
70 done!
71 done!
72 done!
73 done!
74 done!
75 done!
76 done!
77 done!
78 done!
79 done!
80 done!
81 done!
82 done!
83 done!
84 done!
85 done!
86 done!
87 done!
88 done!
89 done!
90 done!
91 done!
92 done!
93 done!
94 done!
95 done!
96 done!
97 done!
98 done!
99 done!
100 done!
101 done!
102 done!
103 done!
104 done!
105 done!
106 done!
107 done!
108 done!
109 done!
110 done!


In [16]:
plt.figure(figsize = (100, 100))
plt.scatter([pos[0]*5000 for pos in positions], [pos[1]*5000 for pos in positions])
for i, txt in enumerate(corpus.vocab):
    plt.annotate(txt, (positions[i][0] * 5000, positions[i][1] * 5000))
plt.show()